Serializer Demonstration -
This Jupyter notebook expects there to be a database available. Remember to run migrations!

Additionally, djangorestframework-xml must be installed for this to work correctly. We do not use the package in the rest of the code, so manual installation is required.

pip install djangorestframework-xml

In [3]:
!pip install djangorestframework-xml

In [4]:
!pip install defusedxml

In [5]:
import django
django.setup()

from datetime import date
from pprint import pprint
from xml.dom.minidom import parseString as xml_parse

from rest_framework.renderers import JSONRenderer
from rest_framework_xml.renderers import XMLRenderer

from organizer.models import NewsLink, Startup, Tag
from organizer.serializers import StartupSerializer, TagSerializer, NewsLinkSerializer

In [7]:
new_tag = Tag.objects.create(name='django2')

In [8]:
s_tag = TagSerializer(new_tag)

In [9]:
s_tag

TagSerializer(<Tag: django2>):
    id = IntegerField(read_only=True)
    name = CharField(max_length=31)
    slug = SlugField(allow_blank=True, max_length=31)

In [10]:
s_tag.data

{'id': 8, 'name': 'django2', 'slug': 'django2'}

In [11]:
JSONRenderer().render(s_tag.data)

b'{"id":8,"name":"django2","slug":"django2"}'

In [12]:
def render_json(serialized_object):
    """Shortcut to make this notebook easier to read"""
    print(
        JSONRenderer().render(
            serialized_object.data,
            accepted_media_type='application/json; indent=4',
        ).decode('utf8')
    )

In [13]:
render_json(s_tag)

{
    "id": 8,
    "name": "django2",
    "slug": "django2"
}


In [14]:
XMLRenderer().render(s_tag.data)

'<?xml version="1.0" encoding="utf-8"?>\n<root><id>8</id><name>django2</name><slug>django2</slug></root>'

In [15]:
def render_xml(serialized_object):
    """Shortcut to make this notebook easier to read
    
    If you need serious XML handling, compare 
    LXML to Python's std-lib XML capabilities.
    """
    print(
        xml_parse(  # python std-lib
            XMLRenderer().render( # Django Rest Framework
                serialized_object.data
            )  
        ).toprettyxml()
    )

In [16]:
render_xml(s_tag)

<?xml version="1.0" ?>
<root>
	<id>8</id>
	<name>django2</name>
	<slug>django2</slug>
</root>



Serialize Object with Relationships

In [18]:
jambon = Startup.objects.create(
    name='JamBon Software1',
    slug='jambon-software1',
    description='Software Consulting & Training for Web and Mobile Products',
    founded_date=date(2013, 1, 18),
    contact='django@jambonsw.com',
    website='https://www.jambonsw.com',
)

In [20]:
jambon.tags.add(new_tag, Tag.objects.create(name='web2'))

In [21]:
s_jambon = StartupSerializer(jambon)

In [22]:
s_jambon

StartupSerializer(<Startup: JamBon Software1>):
    id = IntegerField(read_only=True)
    name = CharField(max_length=31)
    slug = SlugField(max_length=31)
    description = CharField()
    founded_date = DateField()
    contact = EmailField()
    website = URLField(max_length=255)
    tags = TagSerializer(many=True):
        id = IntegerField(read_only=True)
        name = CharField(max_length=31)
        slug = SlugField(allow_blank=True, max_length=31)

In [23]:
s_jambon.data

{'id': 3, 'name': 'JamBon Software1', 'slug': 'jambon-software1', 'description': 'Software Consulting & Training for Web and Mobile Products', 'founded_date': '2013-01-18', 'contact': 'django@jambonsw.com', 'website': 'https://www.jambonsw.com', 'tags': [OrderedDict([('id', 8), ('name', 'django2'), ('slug', 'django2')]), OrderedDict([('id', 10), ('name', 'web2'), ('slug', 'web2')])]}

In [24]:
pprint(s_jambon.data)

{'contact': 'django@jambonsw.com',
 'description': 'Software Consulting & Training for Web and Mobile Products',
 'founded_date': '2013-01-18',
 'id': 3,
 'name': 'JamBon Software1',
 'slug': 'jambon-software1',
 'tags': [OrderedDict([('id', 8), ('name', 'django2'), ('slug', 'django2')]),
          OrderedDict([('id', 10), ('name', 'web2'), ('slug', 'web2')])],
 'website': 'https://www.jambonsw.com'}


In [25]:
render_json(s_jambon)

{
    "id": 3,
    "name": "JamBon Software1",
    "slug": "jambon-software1",
    "description": "Software Consulting & Training for Web and Mobile Products",
    "founded_date": "2013-01-18",
    "contact": "django@jambonsw.com",
    "website": "https://www.jambonsw.com",
    "tags": [
        {
            "id": 8,
            "name": "django2",
            "slug": "django2"
        },
        {
            "id": 10,
            "name": "web2",
            "slug": "web2"
        }
    ]
}


In [26]:
render_xml(s_jambon)

<?xml version="1.0" ?>
<root>
	<id>3</id>
	<name>JamBon Software1</name>
	<slug>jambon-software1</slug>
	<description>Software Consulting &amp; Training for Web and Mobile Products</description>
	<founded_date>2013-01-18</founded_date>
	<contact>django@jambonsw.com</contact>
	<website>https://www.jambonsw.com</website>
	<tags>
		<list-item>
			<id>8</id>
			<name>django2</name>
			<slug>django2</slug>
		</list-item>
		<list-item>
			<id>10</id>
			<name>web2</name>
			<slug>web2</slug>
		</list-item>
	</tags>
</root>



In [27]:
nl = NewsLink.objects.create(
    title='JamBon Software rated best ever',
    slug='jambon-best',
    pub_date=date(2018,4,1),
    link='https://www.xkcd.com/353/',
    startup=jambon,
)

In [28]:
s_nl = NewsLinkSerializer(nl)

In [29]:
s_nl

NewsLinkSerializer(<NewsLink: JamBon Software1:JamBon Software rated best ever>):
    id = IntegerField(read_only=True)
    title = CharField(max_length=63)
    slug = SlugField(max_length=63)
    pub_date = DateField()
    link = URLField(max_length=255)
    startup = StartupSerializer():
        id = IntegerField(read_only=True)
        name = CharField(max_length=31)
        slug = SlugField(max_length=31)
        description = CharField()
        founded_date = DateField()
        contact = EmailField()
        website = URLField(max_length=255)
        tags = TagSerializer(many=True):
            id = IntegerField(read_only=True)
            name = CharField(max_length=31)
            slug = SlugField(allow_blank=True, max_length=31)

In [30]:
pprint(s_nl.data)

{'id': 3,
 'link': 'https://www.xkcd.com/353/',
 'pub_date': '2018-04-01',
 'slug': 'jambon-best',
 'startup': OrderedDict([('id', 3),
                         ('name', 'JamBon Software1'),
                         ('slug', 'jambon-software1'),
                         ('description',
                          'Software Consulting & Training for Web and Mobile '
                          'Products'),
                         ('founded_date', '2013-01-18'),
                         ('contact', 'django@jambonsw.com'),
                         ('website', 'https://www.jambonsw.com'),
                         ('tags',
                          [OrderedDict([('id', 8),
                                        ('name', 'django2'),
                                        ('slug', 'django2')]),
                           OrderedDict([('id', 10),
                                        ('name', 'web2'),
                                        ('slug', 'web2')])])]),
 'title': 'JamBon Software ra